<a href="https://colab.research.google.com/github/Lama-Alfaifi/fraud-detection-anomaly-ai/blob/main/02_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import output

In [16]:
%pip install kagglehub
%pip install datetime
%pip install geopy
output.clear()

In [17]:
import numpy as np
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from geopy.distance import geodesic

In [19]:
path = kagglehub.dataset_download("kartik2112/fraud-detection")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fraud-detection' dataset.
Path to dataset files: /kaggle/input/fraud-detection


In [20]:
df = pd.read_csv(path + '/fraudTrain.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [21]:
drop_col = ['Unnamed: 0', 'cc_num', 'first', 'last', 'trans_num', 'street', 'zip', 'job']
df_clean = df.drop(columns=drop_col)
print('Columns droppe successfully.')
df_clean.info()

Columns droppe successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 15 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1296675 non-null  object 
 1   merchant               1296675 non-null  object 
 2   category               1296675 non-null  object 
 3   amt                    1296675 non-null  float64
 4   gender                 1296675 non-null  object 
 5   city                   1296675 non-null  object 
 6   state                  1296675 non-null  object 
 7   lat                    1296675 non-null  float64
 8   long                   1296675 non-null  float64
 9   city_pop               1296675 non-null  int64  
 10  dob                    1296675 non-null  object 
 11  unix_time              1296675 non-null  int64  
 12  merch_lat              1296675 non-null  float64
 13  merch_long             1296675 non-null  fl


*  **Reason for Dropping Features:**


*   **Privacy & Overfitting:** Removed personal identifiers (first, last, job, street) to protect privacy and prevent the model from memorizing specific individuals instead of learning general patterns.

*   **Redundant IDs:** cc_num and trans_num are unique IDs that don't provide predictive value for fraud behavior.

*  **Simplified Location:** zip is redundant as we use lat and long for more accurate geographic analysis.
*   **Technical Noise:** Unnamed: 0 is just a row index and carries no information.


In [22]:
df_clean['trans_date_trans_time'] = pd.to_datetime(df_clean['trans_date_trans_time'])
df_clean['hour'] =df_clean['trans_date_trans_time'].dt.hour
df_clean['day of week'] =df_clean['trans_date_trans_time'].dt.day_of_week

In [23]:
df_clean = df_clean.drop(columns=['trans_date_trans_time'], errors='ignore')
df_clean.sample(10)

,merchant,category,amt,gender,city,state,lat,long,city_pop,dob,unix_time,merch_lat,merch_long,is_fraud,hour,day of week
966141,fraud_Klein Group,entertainment,79.72,F,Birmingham,IA,40.8626,-91.9534,888,1988-03-25,1359122778,41.604331,-92.744782,0,14,5
749312,fraud_Fritsch and Sons,health_fitness,65.88,M,Saint Petersburg,FL,27.7898,-82.7243,341043,1989-02-28,1353107160,27.069349,-82.471522,0,23,5
639843,fraud_Effertz LLC,entertainment,33.25,M,Wales,AK,64.7556,-165.6723,145,1939-11-09,1348925591,64.309918,-165.026095,0,13,6
245379,fraud_Friesen Inc,shopping_pos,7.28,M,Dallas,TX,32.6900,-96.9177,1263321,1952-09-27,1336035479,33.509843,-96.717992,0,8,4
251679,fraud_Padberg-Rogahn,entertainment,98.38,M,Espanola,NM,35.9866,-106.0654,18408,1972-07-18,1336245084,35.681834,-106.595952,0,19,6
837427,fraud_Kulas Group,health_fitness,70.71,M,Broomfield,CO,39.8854,-105.1139,92337,1967-10-18,1355348813,40.349247,-105.236627,0,21,3
93431,"fraud_Baumbach, Hodkiewicz and Walsh",shopping_pos,2.41,F,Milford,OH,39.1657,-84.2330,31394,1996-11-12,1330126166,38.943752,-84.936084,0,23,6
116443,fraud_Heathcote LLC,shopping_net,9.41,M,Paint Rock,AL,34.7031,-86.3068,653,1966-06-19,1331085028,34.222401,-85.424041,0,1,3
643061,"fraud_Wuckert, Wintheiser and Friesen",home,102.13,F,Wichita,KS,37.6223,-97.3136,409656,1930-10-21,1349013535,36.728359,-96.442815,0,13,0
142609,fraud_Schumm PLC,shopping_net,468.54,F,Cape Coral,FL,26.6939,-81.9452,156391,1968-02-09,1332063285,26.932547,-81.387264,0,9,0


In [24]:
def calculate_distance(row):

 # Calculate geographical distance between customer and merchant
  customer_location = (row['lat'], row['long'])
  merch_location = (row['merch_lat'], row['merch_long'])

  return geodesic(customer_location, merch_location).km


In [25]:
df_clean['distance_km'] = df_clean.apply(calculate_distance, axis=1) # Apply the distance function to each row of the dataset

print('Distance calculation completed')

df_clean[['lat', 'merch_lat', 'distance_km']].head()


Distance calculation completed


,lat,merch_lat,distance_km
0,36.0788,36.011293,78.773821
1,48.8878,49.159047,30.216618
2,42.1808,43.150704,108.102912
3,46.2306,47.034331,95.685115
4,38.4207,38.674999,77.702395


In [26]:
df_clean = df_clean.drop(columns=['merch_lat', 'merch_long'], errors='ignore')
df_clean.head()

,merchant,category,amt,gender,city,state,lat,long,city_pop,dob,unix_time,is_fraud,hour,day of week,distance_km
0,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,Moravian Falls,NC,36.0788,-81.1781,3495,1988-03-09,1325376018,0,0,1,78.773821
1,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,Orient,WA,48.8878,-118.2105,149,1978-06-21,1325376044,0,0,1,30.216618
2,fraud_Lind-Buckridge,entertainment,220.11,M,Malad City,ID,42.1808,-112.2620,4154,1962-01-19,1325376051,0,0,1,108.102912
3,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,Boulder,MT,46.2306,-112.1138,1939,1967-01-12,1325376076,0,0,1,95.685115
4,fraud_Keeling-Crist,misc_pos,41.96,M,Doe Hill,VA,38.4207,-79.4629,99,1986-03-28,1325376186,0,0,1,77.702395
